In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import catboost as cb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501982


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
99864,1112373509,MVO41jK,1,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
12766816,1111654095,8kXrmx,0,MASC,50.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
10882280,1112414519,jkjwOOb,0,FEM,29.0,-,-,-,-,-,...,False,True,False,False,False,False,False,True,False,NaN
3729965,1112289583,Bm5kawQ,1,MASC,23.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN
13463849,1112281044,1o9JX3,0,FEM,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
13181080,1112432239,xkdROxb,0,FEM,20.0,-,-,-,-,-,...,False,False,False,True,False,False,False,False,False,NaN
12004885,1112408816,Yj522bq,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,NaN
15729882,1112237198,owjJNL0,0,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
7064370,1112263416,xkao5lb,0,FEM,20.0,-,-,-,-,Graduado,...,False,False,True,False,False,True,False,True,False,NaN
1583305,1112463452,96Z3q48,0,MASC,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
3667127,1800580,NQxpD,0,MASC,42.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
1477243,1112405448,bOJbbBV,1,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.040000
7535092,1112291790,8MrRZ1M,0,MASC,25.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
9875507,1112474969,ZDwe6kV,0,FEM,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
4730443,1112341049,MVP0b1L,1,MASC,25.0,-,En Curso,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
1326413,1112165615,0zdbao9,0,MASC,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,0.000000
567406,1112335938,owzVKW0,1,FEM,21.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
484329,1112260584,mzbAxmZ,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,0.000000
1134157,1112312684,GNJYeV8,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,True,False,True,True,False,False,False,0.000000
15256870,1112495058,zvPZrMJ,0,FEM,23.0,-,-,-,-,Graduado,...,False,True,True,False,False,False,False,True,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0
train=0
test=0

Train:  14477543 Test:  1608616


In [16]:
gc.collect()

300

In [17]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=7000,max_depth=18,n_estimators=10,colsample_bytree=0.9,n_jobs=-1,random_state=0,silent=False,subsample=1,sumsample_freq=1)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.689839
[2]	valid_0's binary_logloss: 0.685564
[3]	valid_0's binary_logloss: 0.681207
[4]	valid_0's binary_logloss: 0.677747
[5]	valid_0's binary_logloss: 0.673526
[6]	valid_0's binary_logloss: 0.669861
[7]	valid_0's binary_logloss: 0.665884
[8]	valid_0's binary_logloss: 0.662177
[9]	valid_0's binary_logloss: 0.659352
[10]	valid_0's binary_logloss: 0.65637


In [18]:
score=c.score(x_test,y_test)*100
print(score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


80.83352397340322


In [19]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/light.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/light.pkl']

![title](img/grid_lightgbm.png)

In [20]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
39871,41319,1112420745,xkvLm3K,MASC,27.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
29150,75269,1112464014,dY9vLkq,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,True,0.003745
3646,82525,1112466202,6EkJZl,FEM,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.208955
33375,84394,1112466554,owzkvqj,MASC,26.0,-,-,-,-,-,...,True,False,False,False,False,False,False,False,False,0.014085
95734,95218,1112487910,YOb3xr,MASC,31.0,-,-,-,-,-,...,False,False,False,False,False,True,False,False,False,0.000000
62781,10412,1111911438,6jZXbO,MASC,36.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
44261,41824,1112421996,ZNoNVb,FEM,40.0,-,-,-,Abandonado,Graduado,...,False,False,False,False,False,True,False,False,False,0.000000
34090,57541,1112451494,EEMLRJ,MASC,50.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
82679,31070,1112383526,12zXW3,MASC,39.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,True,False,True,0.000000
63333,87590,1112466853,1Z4N3O,FEM,37.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000


In [21]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.52772081, 0.47227919],
       [0.52753657, 0.47246343],
       [0.52665231, 0.47334769],
       ...,
       [0.53653684, 0.46346316],
       [0.53141408, 0.46858592],
       [0.5408177 , 0.4591823 ]])

In [22]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [23]:
prediccion['sepostulo']=y_final2

In [24]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.472279
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.472463
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.473348
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.462802
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.462751


In [25]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.472279
1,1,0.472463
2,2,0.473348
3,3,0.462802
4,4,0.462751


In [26]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/light.csv",index=False)

In [27]:
prediccion.sepostulo.mean()

0.500855751625004